In [1]:
from itertools import cycle
import numpy as np
from scipy.integrate import solve_ivp
from ipywidgets import Button, Checkbox, HBox, VBox, HTMLMath

import plotlymath as pm
from myutils import interact, latex


In [2]:
pm.set_defaults(margins=(40))
colors = pm.plotly.colors.DEFAULT_PLOTLY_COLORS

In [3]:
def stability_text(jacobian):
    trace = np.trace(jacobian)
    det = np.linalg.det(jacobian)
    if det < 0:
        return "Saddle point"
    if trace == 0 or det == 0:
        return "Unknown"
    if trace**2 - 4*det < 0:
        return "Stable spiral" if trace < 0 else "Unstable spiral"
    return "Sink" if trace < 0 else "Source"

def stability_color(jacobian):
    trace = np.trace(jacobian)
    det = np.linalg.det(jacobian)
    if det < 0:
        return "darkorange"
    if trace == 0 or det == 0:
        return "gray"
    return "darkgreen" if trace < 0 else "darkred"

def clickable_phase_portrait(f, xrange, yrange, tmax, **options):
    axes_labels = options.get("axes_labels", (r"$x$", r"$y$"))
    fixed_points = options.get("fixed_points", ())
    jacobian = options.get("jacobian", None)
    fixed_points_color = [stability_color(jacobian(0, pt)) 
            for pt in fixed_points] if jacobian else "black"
    fixed_points_color = options.get("fixed_points_color", fixed_points_color)
    fixed_points_text = [stability_text(jacobian(0, pt)) 
            for pt in fixed_points] if jacobian else ""
    vector_field_color = options.get("vector_field_color", "limegreen")
    vector_field_opacity = options.get("vector_field_opacity", 0.6)
    colors = options.get("colors", pm.plotly.colors.DEFAULT_PLOTLY_COLORS)

    figure, plot = pm.make_figure(widget=True)
    figure.layout.hovermode = "closest"
    figure.layout.update(width=750, height=500)
    plot.axes_labels(*axes_labels)
    plot.axes_ranges(xrange, yrange)
    xmin, xmax = xrange
    ymin, ymax = yrange
    x = np.linspace(xmin, xmax, 51)
    y = np.linspace(ymin, ymax, 51)
    xy = np.moveaxis(np.meshgrid(x, y), 0, 2).reshape(-1, 2)
    plot.points(xy, size=15, opacity=0, hoverinfo="none", showlegend=False, id="grid")

    if vector_field_color:
        plot.vector_field(lambda x, y: f(0, (x, y)), xrange, yrange, 
                color=vector_field_color, opacity=vector_field_opacity, 
                name="Vector field", visible="legendonly", hoverinfo="skip")
    if fixed_points:
        plot.points(fixed_points, color=fixed_points_color, size=10, 
                name="Fixed points", visible="legendonly", hoverinfo="x+y+text", 
                hovertext=fixed_points_text)

    color = cycle(colors)
    options = dict(hoverinfo="skip", showlegend=False, id="solutions[]")
    solve_options = dict(method="RK45", dense_output=True)
    #if jacobian:
    #    solve_options["jac"] = jacobian
    def click_handler(trace, points, state):
        if not (points.xs and points.ys):
            return
        initial_state = (points.xs[0], points.ys[0])
        solution = solve_ivp(f, (0, tmax), initial_state, **solve_options)
        plot.parametric(solution.sol, (0, tmax), color=next(color), **options)
    plot["grid"].on_click(click_handler)

    def clear_figure(widget):
        nonlocal color
        color = cycle(colors)
        with figure.batch_update():
            try:
                del plot["solutions[]"]
            except KeyError:
                pass
    clear_button = Button(description="Clear")
    clear_button.on_click(clear_figure)

    return VBox((clear_button, figure))


In [4]:
def vector_field_linearization(f, xrange, yrange, fixed_points, jacobian, **options):
    zoom_factors = options.get("zoom_factor", (2, 4, 10, 100))
    axes_labels = options.get("axes_labels", (r"$x$", r"$y$"))
    vector_field_color = options.get("vector_field_color", "limegreen")
    vector_field_opacity = options.get("vector_field_opacity", 0.6)
    linearization_color = options.get("linearization_color", "magenta")
    linearization_opacity = options.get("linearization_opacity", 0.6)
    plot_points = options.get("plot_points", 15)

    figure, plot = pm.make_figure(widget=True)
    figure.layout.dragmode = "select" # For this figure, we want select, not pan
    figure.layout.update(width=750, height=500)
    figure.layout.legend.update(x=0.82, xanchor="left")
    plot._subplot.xaxis.domain = (0, 0.8)
    plot.axes_labels(*axes_labels)
    plot.points(fixed_points, color="black", size=10, 
            name="Fixed points", id="fixed points")
    plot.text("", (0, 0), paper=True, visible=False, id="matrix")
    plot.text("", (0, 0), paper=True, visible=False, id="eigenvalues")

    def zoomin(widget):
        selected = []
        if plot["fixed points"].selectedpoints:
            for i in plot["fixed points"].selectedpoints:
                selected.append(fixed_points[i])
        if len(selected) != 1:
            return
        x0, y0 = selected[0]
        J0 = jacobian(0, (x0, y0))
        linearization = lambda x, y: J0 @ (x - x0, y - y0)
        eigenvalues = np.linalg.eigvals(J0)
        if eigenvalues[0].imag:
            eigenvalues = latex(eigenvalues[0], round=3, conjpair=True)
        else:
            eigenvalues = ", ".join([latex(a, round=3) for a in eigenvalues])
        zoom_factor = button_zooms[widget]
        zoom_xrange = (xrange[1] - xrange[0]) / zoom_factor
        zoom_yrange = (yrange[1] - yrange[0]) / zoom_factor
        new_xrange = (x0 - zoom_xrange/2, x0 + zoom_xrange/2)
        new_yrange = (y0 - zoom_yrange/2, y0 + zoom_yrange/2)
        with figure.batch_update():
            plot.axes_ranges(new_xrange, new_yrange)
            plot.vector_field(lambda x, y: f(0, (x, y)), new_xrange, new_yrange, 
                    plotpoints=plot_points, id="vector field")
            plot.vector_field(linearization, new_xrange, new_yrange, 
                    plotpoints=plot_points, 
                    name="Linearization", id="linearization", showlegend=True, 
                    color=linearization_color, opacity=linearization_opacity)
            plot.text(f"${latex(J0, round=4)}$", (0.85, 0.65), paper=True, 
                    visible=True, id="matrix")
            plot.text(f"${eigenvalues}$", (0.85, 0.40), paper=True, 
                    visible=eigenvalues_checkbox.value, id="eigenvalues")

    def zoomout(widget):
        plot["fixed points"].selectedpoints = None
        with figure.batch_update():
            plot.axes_ranges(xrange, yrange)
            plot.vector_field(lambda x, y: f(0, (x, y)), xrange, yrange, 
                    plotpoints=plot_points, 
                    color=vector_field_color, opacity=vector_field_opacity, 
                    name="Vector field", id="vector field")
            plot["linearization"] = pm.points([(0,0)], 
                    visible="legendonly", showlegend=False)
            plot["matrix"].update(visible=False)
            plot["eigenvalues"].update(visible=False)

    def show_eigenvalues(state):
        plot["eigenvalues"].update(visible=state["new"])

    button = Button(description="Zoom out")
    zoomout(button)
    button.on_click(zoomout)
    button_zooms = {button: None}
    for zoom_factor in zoom_factors:
        button = Button(description=f"Zoom in {zoom_factor}x")
        button.on_click(zoomin)
        button_zooms[button] = zoom_factor
    eigenvalues_checkbox = Checkbox(description="Show eigenvalues")
    eigenvalues_checkbox.observe(show_eigenvalues, names="value")
    button_zooms[eigenvalues_checkbox] = None

    return VBox((HBox(tuple(button_zooms.keys())), figure))


In [5]:
def f(t, state):
    x, y = state
    return ( 0.1*x - 0.01*x**2 - 0.04*x*y, 0.08*y - 0.02*x*y - 0.02*y**2 )

def jacobian(t, state):
    x, y = state
    return np.array((
        ( 0.1 - 0.02*x - 0.04*y , -0.04*x                ), 
        ( -0.02*y               , 0.08 - 0.02*x - 0.04*y ), 
    ))

fixed_points = [(0, 0), (10, 0), (0, 4), (2, 2)]

vector_field_linearization(f, (-0.2, 12), (-0.1, 5), fixed_points, jacobian)

In [6]:
def f(t, state):
    x, y = state
    return ( 0.1*x - 0.01*x**2 - 0.04*x*y, 0.08*y - 0.02*x*y - 0.02*y**2 )

def jacobian(t, state):
    x, y = state
    return np.array((
        ( 0.1 - 0.02*x - 0.04*y , -0.04*x                ), 
        ( -0.02*y               , 0.08 - 0.02*x - 0.04*y ), 
    ))

fixed_points = [(0, 0), (10, 0), (0, 4), (2, 2)]

clickable_phase_portrait(f, (-0.05, 12), (-0.02, 5), 500, 
                         fixed_points=fixed_points, jacobian=jacobian)

    'data': [{'hoverinfo': 'no…

In [7]:
r = 0.1
k = 100
e = 0.4
h = 10
b = 0.25
d = 0.08
def f(t, state):
    N, P = state
    return ( r*N*(1 - N/k) - e*N/(h + N)*P, b*e*N/(h + N)*P - d*P )

def jacobian(t, state):
    N, P = state
    return np.array((
        ( r*(1 - 2*N/k) - e*h*P/(h + N)**2 , -e*N/(h + N)      ), 
        ( b*e*h*P/(h + N)**2               , b*e*N/(h + N) - d ), 
    ))

N0 = h / (b*e/d - 1)
P0 = r/e*(1 - N0/k)*(h + N0)
fixed_points = [(0, 0), (k, 0), (N0, P0)]

vector_field_linearization(f, (-2, 110), (-0.4, 15), fixed_points, jacobian)

In [8]:
r = 0.1
k = 100
e = 0.4
h = 10
b = 0.25
d = 0.08
def f(t, state):
    N, P = state
    return ( r*N*(1 - N/k) - e*N/(h + N)*P, b*e*N/(h + N)*P - d*P )

def jacobian(t, state):
    N, P = state
    return np.array((
        ( r*(1 - 2*N/k) - e*h*P/(h + N)**2 , -e*N/(h + N)      ), 
        ( b*e*h*P/(h + N)**2               , b*e*N/(h + N) - d ), 
    ))

N0 = h / (b*e/d - 1)
P0 = r/e*(1 - N0/k)*(h + N0)
fixed_points = [(0, 0), (k, 0), (N0, P0)]

clickable_phase_portrait(f, (-2, 110), (-0.4, 15), 4000, 
                         fixed_points=fixed_points, jacobian=jacobian)

    'data': [{'hoverinfo': 'no…